In [ ]:
#Mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Code
1.   Implement (from scratch) and apply kNN on MNIST with k=1, 5, 10. Apply kNN on raw images, and 2, 7 dimensional eigenspaces (PCA, you can use a library for PCA, or your own code from HW 5), respectively. Show the accuracy scores for each run (you'd run the algorithm 9 times).

2.   Use and run the Random Forest algorithm for MNIST classification (http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Show the accuracy scores and the parameters you used.

If it takes too much time, reduce the number of samples for training/testing (based on random selection).

In [38]:
import six.moves.cPickle as pickle
import gzip
import os
import urllib.request
import numpy as np
import random


# load data
def load_data(dataset):
    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    # Load the dataset
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    print('... data has been loaded!')
    return train_set, valid_set, test_set


#Load the data into train, validation and test sets
train_set, val_set, test_set = load_data('mnist.pkl.gz')

#Separate each set into image vector (_x) and label (_y)
train_x, train_y = train_set
val_x, val_y = val_set
test_x, test_y = test_set

idx = random.sample(range(len(test_x)), 10000)
train_x = [train_x[i] for i in idx]
train_y = [train_y[i] for i in idx]

idx = random.sample(range(len(test_x)), 1000)
test_x = [test_x[i] for i in idx]
test_y = [test_y[i] for i in idx]


Loading data...
... data has been loaded!


In [39]:
# implement kNN
class kNN():
  def __init__(self, k):
    self.k = k

  def euclidean(self, v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

  def fit(self, train_x, train_y):
    self.train_x = train_x
    self.train_y = train_y

  def majority(self, neighbours):
    values, counts = np.unique(neighbours, return_counts=True)
    return values[np.argmax(counts)]

  def get_neighbours(self, test_row):
    distances = list()
    for (train_row, train_class) in zip(self.train_x, self.train_y):
      dist = self.euclidean(train_row, test_row)
      distances.append((dist, train_class))
    distances.sort(key=lambda x: x[0])
    neighbours = list()
    for i in range(self.k):
      neighbours.append(distances[i])
    return neighbours

  def predict(self, test_x):
    preds = []
    for test_row in test_x:
      nearest_neighbours = self.get_neighbours(test_row)
      majority = self.majority(nearest_neighbours)
      preds.append(majority)
    return np.array(preds)


def accuracy(preds, label):
  return 100 * (preds == label).mean()


In [40]:
from sklearn.decomposition import PCA

# kNN experiment
for j in [0,2,7]:
  if j:
    pca = PCA(n_components=j)
    train = pca.fit_transform(train_x)
    test = pca.fit_transform(test_x)
  else :
    train = train_x
    test = test_x
  for i in [1,5,10]:
    knn = kNN(k=i)
    knn.fit(train,train_y)
    result = knn.predict(test)
    if j:
      print(f'(pca={j}, k={i}) accuracy: {accuracy(result, test_y):.3f} %')
    else:
      print(f'(raw, k={i}) accuracy: {accuracy(result, test_y):.3f} %')

(raw, k=1) accuracy: 58.500 %
(raw, k=5) accuracy: 95.200 %
(raw, k=10) accuracy: 94.500 %
(pca=2, k=1) accuracy: 6.300 %
(pca=2, k=5) accuracy: 39.000 %
(pca=2, k=10) accuracy: 40.400 %
(pca=7, k=1) accuracy: 9.600 %
(pca=7, k=5) accuracy: 38.200 %
(pca=7, k=10) accuracy: 39.500 %


In [42]:
from sklearn.ensemble import RandomForestClassifier

# ramdom forest experiment
n_estimators = [50, 100, 200]

for i in range(3):
  randf = RandomForestClassifier(n_estimators=n_estimators[i])
  randf.fit(train_x, train_y)
  result = randf.predict(test_x)
  print(f'(n_estimators={n_estimators[i]}) accuracy: {accuracy(result, test_y):.3f} %')

(n_estimators=50) accuracy: 94.600 %
(n_estimators=100) accuracy: 94.900 %
(n_estimators=200) accuracy: 95.000 %


# Report


1.   What is the goal of classification algorithms?
2.   Compare kNN and Random Forest in terms of bias/variance, statistical assumptions of data (which data do they work best with?), sample complexity (how much data do they need to perform acceptably well), computational complexity, and others if you want.
3.   In your implementation, which algorithm had the highest accuracy? What are some of the factors that contributed to one being better that the other?
4.   In terms of computation time, which algorithm was faster? Why?
5.   Is it always better to choose the algorithm that has the best accuracy or the one that has the fastest computational time? Give yout opinion and some examples of applications.
6.   Conclude with some thoughts and things you learned from this homework.

1. The goal of classification algorithms is to correctly find the class of the input so that data can be classified.

2. In terms of bias/variance, kNN has low bias and high variance in low k values; vice versa, random forest has both low bias and low variance. In terms of statistical assumptions of data, kNN is good for data with low dimensionality, and random forest is good for data with high dimensionality. In terms of sample complexity, kNN needs lots of data samples, and random forest needs less data samples. In terms of computational complexity, kNN is simpler than random forest.

3. In my implementation, random forest with n_estimators=200 showed the highest accuracy. In my case, a larger n_estimators value was the key to high accuracy.

4. In terms of computation time, the random forest was faster than kNN. The most significant reason is the different methods of predicting. For kNN, it computes every distance when it starts predicting, but random forest computes most of the things before predicting when it makes the trees. There might be some optimization issues as well because kNN is implemented from scratch, but random forest is implemented from the sklearn library.

5. In machine learning, there's no right answer that works for every situation. There are many dependencies that should be considered when choosing the algorithms.

6. I've learned that the computational resources of the algorithm, like time and space, are really important for machine learning because it should be able to handle a large amount of data.
